In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

df = pd.read_csv(r"C:\Users\A.SAIRAM\Downloads\81 column dataset.csv")

print(df.info())
print(df.describe().transpose())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [19]:
x =df .drop(columns=['Id','SalePrice'],axis=1)
y = df ['SalePrice']


# # preprocessing for numerical and categorical features
# # numerical_feature = x.select_dtypes(include=['int64','float64'])
# # categorical_feature = x.select_dtypes(include=['object']).columns
numerical_feature = ['LotArea', 'GrLivArea', 'TotalBsmtSF']  # Example columns; replace with actual
categorical_feature = ['MSZoning', 'Street', 'Neighborhood'] 


#Create preprocessing pipelines
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy ='median')),
    ('scaler',StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown= 'ignore'))
])

preprocessor = ColumnTransformer([
    ('num',num_pipeline,numerical_feature),
    ('cat',cat_pipeline,categorical_feature)
])

#fit and transform the data
x_processed = preprocessor.fit_transform(x)

In [36]:
model = Sequential ([
    Dense(128, activation ='relu', input_shape =( x_processed.shape[1],)),
    Dropout(0.3),
    Dense(64,activation = 'relu',),
    Dropout(0.3),
    Dense(32, activation = 'relu'),
    Dense(1) # output layer for regression
])

# compile the model
model.compile(optimizer =Adam(learning_rate= 0.001),
              loss='mean_squared_error',
              metrics = ['mean_squared_error'])

In [29]:
# x =df .drop(columns=['Id','SalePrice'],axis=1)
# y = df ['SalePrice']
from sklearn.model_selection import train_test_split
# spilt the data into training and validation sets
x_train,x_val,y_train,y_val= train_test_split(x_processed, y, test_size =0.2, random_state =42)

# Train the model
history = model.fit(x_train, y_train,
                    epochs= 100,
                    validation_data=(x_val,y_val),
                    verbose = 1)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 36759969792.0000 - mean_squared_error: 36759969792.0000 - val_loss: 39651213312.0000 - val_mean_squared_error: 39651213312.0000
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 37405487104.0000 - mean_squared_error: 37405487104.0000 - val_loss: 39619338240.0000 - val_mean_squared_error: 39619338240.0000
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 39464955904.0000 - mean_squared_error: 39464955904.0000 - val_loss: 39422210048.0000 - val_mean_squared_error: 39422210048.0000
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 38940811264.0000 - mean_squared_error: 38940811264.0000 - val_loss: 38706724864.0000 - val_mean_squared_error: 38706724864.0000
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 38274097152.0000 - mean_squared_error: 38274097152.0000 - val_loss: 36862402560.0000 - val_mean_squared_error: 36862402560.0000
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 34526

In [30]:
val_loss, val_mse = model.evaluate(x_val, y_val)
print(f'validation Loss: {val_loss}')
print(f'validation mean squared error: {val_mse}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1569740288.0000 - mean_squared_error: 1569740288.0000
validation Loss: 1611378688.0
validation mean squared error: 1611378688.0
